# 데이터 불러오기

In [2]:
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 경로 설정       
os.chdir('/content/drive/My Drive/ESAA 데이터 분석 프로젝트 (가짜 뉴스 분류)/Data')
os.getcwd()

'/content/drive/My Drive/ESAA 데이터 분석 프로젝트 (가짜 뉴스 분류)/Data'

In [5]:
import pandas as pd
train = pd.read_csv("news_train.csv")
test = pd.read_csv("news_test.csv")
train.head()

,n_id,date,title,content,ord,info
0,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도,1,0
1,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"""실적기반"" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개",2,1
2,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시",3,1
3,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지,4,0
4,NEWS09727,20200626,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국적인 소비 붐 조성에 기여할 예정,1,0


# 최종모델

정확도 : 0.97297

In [7]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 460kB 35.9MB/s 
     |████████████████████████████████| 92kB 8.1MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [8]:
from konlpy.tag import Komoran
import re
import tqdm
komoran  = Komoran()
 
def text_preprocessing(text_list):
    
    stopwords = ['을', '를', '이', '가', '은', '는', 'null']
    tokenizer = Komoran()
    token_list = []
    
    for text in tqdm.tqdm(text_list):
      try:
        txt = re.sub('[^가-힣a-z]', ' ', text.lower()) 
        # 한글과 영어 소문자만 남기고 다른 글자 모두 제거
        token = tokenizer.morphs(txt) # 형태소 분석
        token = [t for t in token if t not in stopwords or type(t) != float] 
        #형태소 분석 결과 중 stopwords에 해당하지 않는 것만 추출
        token_list.append(token)
      except:
        token_list.append(None)
    return token_list, tokenizer
 
train['new_article'], komoran = text_preprocessing(train['content'])

100%|██████████| 118745/118745 [01:46<00:00, 1117.64it/s]


In [10]:
# train으로 학습시킨 형태소 분석기 test에 적용
stopwords = ['을', '를', '이', '가', '은', '는', 'null']
token_list2 = []
for text in tqdm.tqdm(test['content']):
  try:
    txt2 = re.sub('[^가-힣a-z]', ' ', text.lower())
    token2 = komoran.morphs(txt2)
    token2 = [t for t in token2 if t not in stopwords or type(t) != float]
    token_list2.append(token2)
  except:
    token_list2.append(None)
test['new_article'] = token_list2

100%|██████████| 142565/142565 [01:59<00:00, 1192.33it/s]


In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
train_notnull = train[train['new_article'].notnull()]
 
def text2sequence(train_text, max_len=1000):
    tokenizer = Tokenizer() #keras의 vectorizing 함수 호출
    tokenizer.fit_on_texts(train_text) #train 문장에 fit
    train_X_seq = tokenizer.texts_to_sequences(train_text) #각 토큰들에 정수 부여
    vocab_size = len(tokenizer.word_index) + 1 #모델에 알려줄 vocabulary의 크기 계산
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len) 
    #설정한 문장의 최대 길이만큼 padding
    
    return X_train, vocab_size, tokenizer
 
train_y = train_notnull['info']
train_X, vocab_size, vectorizer = text2sequence(train_notnull['new_article'], 
                                                max_len = 100)
print(train_X.shape, train_y.shape)

vocab_size :  33515
(118665, 100) (118665,)


In [12]:
test_notnull = test[test['new_article'].notnull()]
test_X_seq = vectorizer.texts_to_sequences(test_notnull['new_article'])
test_X = pad_sequences(test_X_seq, maxlen = 100)

In [13]:
import gensim
import numpy as np
 
word2vec = gensim.models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin.gz', binary = True)
embedding_matrix = np.zeros((vocab_size, 300))
for index, word in enumerate(vectorizer.word_index):
    if word in word2vec:
        embedding_vector = word2vec[word] 
        embedding_matrix[index] = embedding_vector 
    else:
        print("word2vec에 없는 단어입니다.")
        continue

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word2vec에 없는 단어입니다.
word

In [15]:
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Embedding, Dropout, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
 
model = Sequential()
model.add(Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = 1000))
model.add(Dropout(0.2))
model.add(Conv1D(32, 5, strides=1, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
 
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 3)
mc = ModelCheckpoint('best_model.h5', monitor = 'val_acc', mode = 'max', 
                     verbose = 1, save_best_only = True)
history = model.fit(train_X, train_y, epochs = 10, batch_size=64, 
                    validation_split=0.2, callbacks=[es, mc])
 
pred_test = model.predict(test_X)
test_notnull.loc[:,'info'] = np.where(pred_test> 0.5, 1,0).reshape(-1)
test_notnull['id'] = test_notnull['n_id']+"_"+test_notnull['ord'].map(str)
test_notnull.drop(['n_id', 'ord'], axis='columns', inplace=True)
 
test['id'] = test['n_id']+"_"+test['ord'].map(str)
test.drop(['n_id', 'ord'], axis='columns', inplace=True)
merge = pd.merge(test, test_notnull, how='outer', on='id')
merge = merge.loc[:,['id', 'info']]
merge

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 300)         10054500  
_________________________________________________________________
dropout (Dropout)            (None, 1000, 300)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 996, 32)           48032     
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                2112      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

KeyboardInterrupt: ignored

In [ ]:
# train에서 발견한 결과 적용하여 결측치 채우기
 
merge.fillna(0, inplace=True) # 숫자들은 모두 0
merge.at[29937, 'info']=1
merge.at[31326, 'info']=1 
# test 셋에서 확인 결과 이 두 인덱스의 content만 ‘]]’였음 (train에서 이를 1로 판별)